In [2]:
from agent.dspy_agent import OpenBBAgentChroma
from agent.database import load_database, build_database, build_docs_metadata, build_graph
from dotenv import load_dotenv,find_dotenv
import os
load_dotenv(find_dotenv(),override=True)

True

In [3]:
openbb_graph,router_names = build_graph()
docs,metadata = build_docs_metadata(openbb_graph)

In [4]:
len(docs),len(metadata)

(337, 337)

In [5]:
# build_database(docs,metadata,os.environ['OPENAI_API_KEY'])

In [6]:
openbb_collection = load_database(os.environ['OPENAI_API_KEY'])

In [7]:
obb_chroma = OpenBBAgentChroma(openbb_collection)

In [10]:
funcs,prompts = obb_chroma("What is the cryptocurrency pair of Ethereum")

First level string: crypto: Get historical price data for cryptocurrency pair(s) within a provider. Search available cryptocurrency pairs within a provider. 

currency: Currency Historical Price. Currency historical data.  Currency historical prices refer to the past exchange rates of one currency against another over a specific period. This data provides insight into the fluctuations and trends in the foreign exchange market, helping analysts, traders, and economists understand currency performance, evaluate economic health, and make predictions about future movements. Currency Search.  Search available currency pairs. Currency pairs are the national currencies from two countries coupled for trading on the foreign exchange (FX) marketplace. Both currencies will have exchange rates on which the trade will have its position basis. All trading within the forex market, whether selling, buying, or trading, will take place through currency pairs. (ref: Investopedia) Major currency pairs inc

In [11]:
funcs

[{'ids': ['id152'],
  'embeddings': None,
  'metadatas': [{'function_call': '{\'name\': \'obb-crypto-search_fmp\', \'description\': \'Search available cryptocurrency pairs within a provider.\', \'parameters\': {\'type\': \'object\', \'properties\': {\'query\': {\'type\': \'string\', \'description\': \'Search query.\', \'default\': None}, \'provider\': {\'description\': "The provider to use for the query, by default None. If None, the provider specified in defaults is selected or \'fmp\' if there is no default.", \'default\': \'fmp\', \'type\': \'string\', \'enum\': [\'fmp\']}}, \'required\': [\'provider\']}}',
    'node_name': 'obb-crypto-search_fmp',
    'provider_source': 'fmp',
    'trail': 'crypto-->search',
    'type': 'provider_function'}],
  'documents': ['empty'],
  'uris': None,
  'data': None}]

In [8]:
from openbb import obb
obb.account.login(pat=os.environ['OBB_PAT'])

In [14]:
from typing import List
from agent.function_call import run_function_calling, format_function
def run_main_function_callinng(question:str,provider_list:List[str]=[]):
    funcs,prompts = obb_chroma(question)
    print(funcs)
    provider_sources = [fn['provider_source'] for fn in funcs[0]['metadatas']]
    if provider_list != []:
        valid_providers = [sources for sources in provider_sources if sources in provider_list]
    else:
        valid_providers = provider_sources
    
    return funcs,valid_providers
    # resp = run_function_calling(funcs,valid_providers[0],question)
    # print(resp)
    # final_func = format_function(resp)
    # return final_func

In [15]:
funcs,valid_providers = run_main_function_callinng("I want the futures Gold prices")

First level string: derivatives: Get the complete options chain for a ticker. Historical futures prices. Futures Term Structure, current or historical. 

economy: Get Forecasted GDP Data. Get Nominal GDP Data. Get Real GDP Data. Get the upcoming, or historical, economic calendar of global events. Get Consumer Price Index (CPI).  Returns either the rescaled index value, or a rate of change (inflation). Get Market Risk Premium by country. Search for FRED series or economic releases by ID or string.  This does not return the observation values, only the metadata. Use this function to find series IDs for `fred_series()`. Get data by series ID from FRED. Get Money Measures (M1/M2 and components).  The Federal Reserve publishes as part of the H.6 Release. Get global unemployment data. Use the composite leading indicator (CLI).  It is designed to provide early signals of turning points in business cycles showing fluctuation of the economic activity around its long term potential level.  CLIs 

In [37]:
funcs

[{'ids': ['id163'],
  'embeddings': None,
  'metadatas': [{'function_call': '{\'name\': \'obb-derivatives-futures-historical_yfinance\', \'description\': \'Historical futures prices.\', \'parameters\': {\'type\': \'object\', \'properties\': {\'symbol\': {\'type\': \'string\', \'description\': \'Symbol to get data for. Multiple items allowed for provider(s): yfinance.\', \'default\': \'\'}, \'start_date\': {\'type\': \'string\', \'description\': \'Start date of the data, in YYYY-MM-DD format.\', \'default\': None}, \'end_date\': {\'type\': \'string\', \'description\': \'End date of the data, in YYYY-MM-DD format.\', \'default\': None}, \'expiration\': {\'type\': \'string\', \'description\': \'Future expiry date with format YYYY-MM\', \'default\': None}, \'provider\': {\'description\': "The provider to use for the query, by default None. If None, the provider specified in defaults is selected or \'yfinance\' if there is no default.", \'default\': \'yfinance\', \'type\': \'string\', \'enu

In [38]:
import ast
for func in funcs:
    if func != []:
        for meta in func["metadatas"]:
            if meta["provider_source"] == valid_providers[0]:
                function_call = ast.literal_eval(meta["function_call"])
                function_call["name"] = function_call["name"].rpartition("_")[0]
                break

In [39]:
function_call

{'name': 'obb-derivatives-futures-historical',
 'description': 'Historical futures prices.',
 'parameters': {'type': 'object',
  'properties': {'symbol': {'type': 'string',
    'description': 'Symbol to get data for. Multiple items allowed for provider(s): yfinance.',
    'default': ''},
   'start_date': {'type': 'string',
    'description': 'Start date of the data, in YYYY-MM-DD format.',
    'default': None},
   'end_date': {'type': 'string',
    'description': 'End date of the data, in YYYY-MM-DD format.',
    'default': None},
   'expiration': {'type': 'string',
    'description': 'Future expiry date with format YYYY-MM',
    'default': None},
   'provider': {'description': "The provider to use for the query, by default None. If None, the provider specified in defaults is selected or 'yfinance' if there is no default.",
    'default': 'yfinance',
    'type': 'string',
    'enum': ['yfinance']},
   'interval': {'description': 'Time interval of the data to return.',
    'default': '1

In [62]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([("human", "{input}"),("system","You can write functions from the given tool. Double check your response with correct parameter names and values. Also, check for any invalid parameter values. Also, write a python function to save the output of the code.")])

model = ChatOpenAI(temperature=0,model="gpt-3.5-turbo").bind(
    functions=[function_call], function_call={"name": function_call["name"]}
)
runnable = prompt | model
resp = runnable.invoke({"input": "I want the futures Gold prices"})

In [63]:
resp

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"symbol":"GC=F","provider":"yfinance"}', 'name': 'obb-derivatives-futures-historical'}}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 333, 'total_tokens': 344}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c9fd82c0-8877-43a7-aa22-ad8567f8db71-0')

In [43]:
def format_function(function_response):
    obb_func = function_response.additional_kwargs["function_call"]
    obb_func_name = obb_func["name"]
    obb_func_name = ".".join(obb_func_name.split("-")) + "("
    args_dict = ast.literal_eval(obb_func["arguments"])
    for arg, val in args_dict.items():
        if isinstance(val,str):
            val = f"'{val}'"
        obb_func_name += f"{arg}={val},"
    obb_func_name = obb_func_name[:-1] + ")"
    final_func = "from openbb import obb\n" + obb_func_name
    return final_func

obb_func = format_function(resp)

In [45]:
python_func = f"""
```python
{obb_func}
```
"""
print(python_func)


```python
from openbb import obb
obb.derivatives.futures.historical(symbol='GC=F',provider='yfinance')
```



In [49]:
obb_func = "from openbb import obb\nobb.derivatives.futures.historical(symbol='GC',provider='yfinance')"

In [50]:
from pathlib import Path

from autogen.coding import CodeBlock, LocalCommandLineCodeExecutor
obb.user.preferences.output_type = "dataframe"

work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)

executor = LocalCommandLineCodeExecutor(work_dir=work_dir)
print(
    executor.execute_code_blocks(
        code_blocks=[
            CodeBlock(language="python", code=obb_func),
        ]
    )
)

exit_code=0 output='' code_file='/home/athekunal/Function Calling/hierarchical-openbb-agent/coding/tmp_code_6e9cc7fb0efbd8f982215483c4db8195.py'


In [48]:
obb.user.preferences.output_type = "dataframe"


,open,high,low,close,volume
date,,,,,
2000-08-30,273.899994,273.899994,273.899994,273.899994,0.0
2000-08-31,274.799988,278.299988,274.799988,278.299988,0.0
2000-09-01,277.000000,277.000000,277.000000,277.000000,0.0
2000-09-05,275.799988,275.799988,275.799988,275.799988,2.0
2000-09-06,274.200012,274.200012,274.200012,274.200012,0.0
...,...,...,...,...,...
2024-05-29,2340.300049,2340.300049,2340.300049,2340.300049,119317.0
2024-05-30,2336.899902,2349.500000,2320.800049,2342.899902,32587.0
2024-05-31,2344.100098,2354.000000,2319.000000,2322.899902,941.0


In [59]:
code_block = CodeBlock(language="python", code=obb_func)

In [60]:
from autogen.coding import CodeBlock, CodeExecutor, CodeExtractor, CodeResult, MarkdownCodeExtractor
from typing import List

from IPython import get_ipython

class NotebookExecutor(CodeExecutor):

    @property
    def code_extractor(self) -> CodeExtractor:
        # Extact code from markdown blocks.
        return MarkdownCodeExtractor()

    def __init__(self) -> None:
        # Get the current IPython instance running in this notebook.
        self._ipython = get_ipython()

    def execute_code_blocks(self, code_blocks: List[CodeBlock]) -> CodeResult:
        log = ""
        for code_block in code_blocks:
            result = self._ipython.run_cell("%%capture --no-display cap\n" + code_block.code)
            log += self._ipython.ev("cap.stdout")
            log += self._ipython.ev("cap.stderr")
            if result.result is not None:
                log += str(result.result)
            exitcode = 0 if result.success else 1
            if result.error_before_exec is not None:
                log += f"\n{result.error_before_exec}"
                exitcode = 1
            if result.error_in_exec is not None:
                log += f"\n{result.error_in_exec}"
                exitcode = 1
            if exitcode != 0:
                break
        return CodeResult(exit_code=exitcode, output=log)


In [61]:
notebook_executor = NotebookExecutor()

notebook_executor.execute_code_blocks([code_block])

,open,high,low,close,volume
date,,,,,
2000-08-30,273.899994,273.899994,273.899994,273.899994,0.0
2000-08-31,274.799988,278.299988,274.799988,278.299988,0.0
2000-09-01,277.000000,277.000000,277.000000,277.000000,0.0
2000-09-05,275.799988,275.799988,275.799988,275.799988,2.0
2000-09-06,274.200012,274.200012,274.200012,274.200012,0.0
...,...,...,...,...,...
2024-05-29,2340.300049,2340.300049,2340.300049,2340.300049,119317.0
2024-05-30,2336.899902,2349.500000,2320.800049,2342.899902,32587.0
2024-05-31,2344.100098,2354.000000,2319.000000,2322.899902,941.0


CodeResult(exit_code=0, output='')

In [ ]:
import ast
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

def format_function(function_response):
    obb_func = function_response.additional_kwargs["function_call"]
    obb_func_name = obb_func["name"]
    obb_func_name = ".".join(obb_func_name.split("-")) + "("
    args_dict = ast.literal_eval(obb_func["arguments"])
    for arg, val in args_dict.items():
        if isinstance(val,str):
            val = f"'{val}'"
        obb_func_name += f"{arg}={val},"
    obb_func_name = obb_func_name[:-1] + ")"
    final_func = "from openbb import obb\n" + obb_func_name
    return final_func

def main_function_calling(question:str,provider_list:List[str]=[]):
    funcs,prompts = obb_chroma(question)
    print(funcs)
    provider_sources = [fn['provider_source'] for fn in funcs[0]['metadatas']]
    if provider_list != []:
        valid_providers = [sources for sources in provider_sources if sources in provider_list]
    else:
        valid_providers = provider_sources
    for func in funcs:
        if func != []:
            for meta in func["metadatas"]:
                if meta["provider_source"] == valid_providers[0]:
                    function_call = ast.literal_eval(meta["function_call"])
                    function_call["name"] = function_call["name"].rpartition("_")[0]
                    break
    
    prompt = ChatPromptTemplate.from_messages([("human", "{input}"),("system","You can write functions from the given tool. Double check your response with correct parameter names and values. Also, check for any invalid parameter values")])

    model = ChatOpenAI(temperature=0,model="gpt-3.5-turbo").bind(
        functions=[function_call], function_call={"name": function_call["name"]}
    )
    runnable = prompt | model
    resp = runnable.invoke({"input": question})

    obb_func = format_function(resp)
    code_block = CodeBlock(language="python", code=obb_func)
    return code_block



notebook_executor = NotebookExecutor()

notebook_executor.execute_code_blocks([code_block])
